In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cdec_maps import cdec

In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts, dim
import geoviews as gv
import panel as pn
import warnings
warnings.filterwarnings('ignore')
import tqdm
import pandas as pd

# Read all stations

In [ ]:
r = cdec.Reader()

stations = r.read_all_stations()

stations

# Display on map

In [ ]:
stations = stations[stations.Latitude > 30]

stations.hvplot.points('Longitude','Latitude',geo=True,tiles='CartoLight',hover_cols='all')

# Read stations with meta data (all in one)

This is cached data and may take a long time to create as it reads all stations and their meta data and then caches it

In [ ]:
stations_meta = cdec.read_stations_with_meta()
stations_meta.head(2)

In [ ]:
stations_meta = stations_meta[stations_meta.Latitude>30]

# Plot all stations with sensor type filtering

In [ ]:
tmap = hv.element.tiles.CartoLight()
tmap*gv.Points(stations, kdims=['Longitude','Latitude'])
def map_sensors(sensor_description):
    df = stations_meta.query(f'`Sensor Description`=="{sensor_description}"')
    return tmap*gv.Points(df, kdims=['Longitude','Latitude'])
    #df.hvplot.points('Longitude','Latitude',geo=True,tiles='CartoLight',hover_cols='all').opts(xlim=(-123,117),ylim=(32,42))

sensor_descs = pn.widgets.Select(options=list(stations_meta['Sensor Description'].unique()))
dmap = hv.DynamicMap(map_sensors, streams={'sensor_description':sensor_descs})
pn.Row(dmap,sensor_descs)

# Plot map of accumulated precip stations with daily duration

In [ ]:
precip_accum_stations = stations_meta.query('`Sensor Description`=="PRECIPITATION, ACCUMULATED, INCHES"')

precip_accum_stations

precip_accum_stations = precip_accum_stations[precip_accum_stations['Data Available'].str.contains('present')]

precip_accum_stations_daily = precip_accum_stations[precip_accum_stations['Duration'].str.contains('daily')]

precip_accum_stations_daily.hvplot.points('Longitude','Latitude',geo=True, tiles='CartoLight',hover_cols='all')

# Get data for all these stations upto current and then plot on map 

In [ ]:

#SAE -> Sacramento Executive Airport

x = {sid: r.read_station_data(sid, 2, 'D', '2022-10-01','') for sid in tqdm.tqdm(precip_accum_stations_daily.ID)}
dfs = [r.read_station_data(sid, 2, 'D', '2022-10-01','2023-01-17') for sid in precip_accum_stations_daily.ID]


In [ ]:
import holoviews as hv
from holoviews import opts, dim

In [ ]:
date='2023-01-17'

In [ ]:
v=[df.VALUE[pd.to_datetime(date)] for df in dfs]
df = pd.DataFrame(v,index=precip_accum_stations_daily.ID).reset_index()
df.columns=['ID','PRECIP_ACCUM']
merged = precip_accum_stations_daily.merge(df,on='ID')


In [ ]:
def show_accum_precip_map(date):
    v=[df.VALUE[pd.to_datetime(date)] for df in dfs]
    df = pd.DataFrame(v,index=precip_accum_stations_daily.ID).reset_index()
    df.columns=['ID','PRECIP_ACCUM']
    merged = precip_accum_stations_daily.merge(df,on='ID')
    plt = tmap*gv.Points(merged, 
               ['Longitude','Latitude']).opts(
        color=dim('PRECIP_ACCUM'),cmap='rainbow4',cnorm='eq_hist',colorbar=True, clim=(0,70),
        size=dim('PRECIP_ACCUM')*0.3,
        frame_width=400)
    return plt

In [ ]:
date_slider = pn.widgets.DateSlider(name='Date Slider', 
                                    start=pd.to_datetime('2022-10-01'), 
                                    end=pd.to_datetime('2023-01-19'), 
                                    value=pd.to_datetime('2023-01-17'))

In [ ]:
pn.Column(date_slider, hv.DynamicMap(show_accum_precip_map, streams = {'date':date_slider}))#.show()